In [1]:
import datetime, json, os, requests, pickle
import pandas as pd
import numpy as np
from datetime import datetime
from konlpy.tag import Okt
from django.conf import settings
from sklearn.metrics.pairwise import cosine_similarity
from math import radians, sin, cos, sqrt, atan2

In [2]:
# 피클 파일을 읽어올 경로와 파일명 설정
file_path = r'/home/lab17/final_project/jeju_olleh/jeju_olleh/modules/tfidf_matrix.pkl'

# 피클 파일 읽어오기
with open(file_path, 'rb') as f:
    tfidf_matrix = pickle.load(f)
# 읽어온 데이터를 사용하여 원하는 작업 수행

In [3]:
# 피클 파일을 읽어올 경로와 파일명 설정
file_path = r'/home/lab17/final_project/jeju_olleh/jeju_olleh/modules/tfidf_vectorizer.pkl'

# 피클 파일 읽어오기
with open(file_path, 'rb') as f:
    tfidf_vectorizer = pickle.load(f)
# 읽어온 데이터를 사용하여 원하는 작업 수행

In [12]:
def get_address_info(address):
    # 주소를 가져오기 위한 url
    apiurl = "https://api.vworld.kr/req/address?"
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "road",
        "key": "054C4365-485B-35C4-9219-1C30E6B61D7C"
    }
    response = requests.get(apiurl, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'response' in data and 'refined' in data['response'] and 'text' in data['response']['refined']:
            address = data['response']['refined']['text']
            latitude = float(data['response']['result']['point']['y'])
            longitude = float(data['response']['result']['point']['x'])
            # DataFrame 생성
            df = pd.DataFrame({
                'Address': [address],
                'Latitude': [latitude],
                'Longitude': [longitude]
            })
            return df
        else:
            print("Failed to retrieve data from the API.")
            return pd.DataFrame()  # 빈 DataFrame 반환
    else:
        print("Failed to retrieve data from the API.")
        return pd.DataFrame()  # 빈 DataFrame 반환

In [5]:
# 두 지점 간의 Haversine 거리를 계산합니다. 
def lat_long_distance(lat1, lon1, lat2, lon2):
    
    # 지구의 반지름 (단위: km)
    R = 6371.0

    # 라디안으로 변환
    lat1 = np.radians(lat1)  # 여기서 np.radians 사용
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)
    
    # 위도와 경도의 차이 계산
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    # Haversine 공식 계산
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # 거리 계산
    distance = R * c
    
    return distance

In [9]:
def tokenize_user_input(user_input):
    okt = Okt()
    # 불용어 설정
    stopwords = pd.read_csv(r'/home/lab17/final_project/jeju_olleh/jeju_olleh/modules/ko-stopwords.csv')
    stopwords = list(stopwords['stopwords'])
    stopwords += [
        '에서', '고', '이다', '는', '한', '씨', '것', '거', '게', '데', '이다', '건', '고', '되다', '되어다', '걸', '기',
        '시', '네', '듯', '랍니', '중이', '얘', '스', '도도', '나', '수', '개', '내', '기', '제', '저', '인', '있다', '이렇다',
        '그렇다', '번', '위', '팅', '분', '인', '링', '란', '포', '두', '진짜', '하다', '이다', '가다', '이제', '들다',
        '너무', '먹다', '제주', '제주도', '있다', '있으며', '경우', '대부분', '조황', '로서', '조황', '있는', '최근', '그대로', '해당', '만큼', '등',
        '있게', '있으므로', '더군다나', '다시', '그', '발생', '가장', '중', '아니다', '위해',
        '후', '곳', '주로', '위해', '많다', '매일', '주로', '때문', '곳', '좋다', '좋은',
        '총', '속', '주', '후', '만', '원', '외', '등', '현재', '매장', '없다', '층', '숍', '별로', '리', '흠뻑', '의하다', '점', '도', '이러하다',
        '있는', '있었다', '비짓', '출처', '중요한', '중요하다', '특별자치도', '제주특별자치도'
    ]
    stopwords = set(stopwords)
    pos_words = okt.pos(user_input, stem=True, norm=True)

    # 형태소 분석 및 불용어 제거
    tokens = [word for word, tag in pos_words if tag in ['Noun', 'Adjective'] and word not in stopwords]
    return tokens

In [7]:
user_input = '아이와 가기 좋은 캠핑장'
user_address = '제주특별자치도 제주시 우도면 우도로 324-1'

In [10]:
keyword_input = tokenize_user_input(user_input)
keyword_input = ' '.join(keyword_input)
print(keyword_input)

가기 캠핑장


In [13]:
user_coordinates = get_address_info(user_address)

ConnectionError: HTTPSConnectionPool(host='api.vworld.kr', port=443): Max retries exceeded with url: /req/address?service=address&request=getcoord&crs=epsg%3A4326&address=%EC%A0%9C%EC%A3%BC%ED%8A%B9%EB%B3%84%EC%9E%90%EC%B9%98%EB%8F%84+%EC%A0%9C%EC%A3%BC%EC%8B%9C+%EC%9A%B0%EB%8F%84%EB%A9%B4+%EC%9A%B0%EB%8F%84%EB%A1%9C+324-1&format=json&type=road&key=054C4365-485B-35C4-9219-1C30E6B61D7C (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f85c11466a0>: Failed to establish a new connection: [Errno 110] Connection timed out'))

In [ ]:
user_latitude = user_coordinates['Latitude'].iloc[0]
user_longitude = user_coordinates['Longitude'].iloc[0]
print(user_coordinates)
print(user_latitude)
print(user_longitude)

In [28]:
df = pd.read_csv(r'C:\Users\gherwt\Desktop\파이널 프로젝트\data1\fin_token.csv')
df.head(5)

,Unnamed: 0,title,firstimage,cat1,cat2,cat3,addr1,mapx,mapy,overview,Rating,Reviews,overview_prep,tokenizing
0,0,가시리국산화풍력발전단지,http://tong.visitkorea.or.kr/cms/resource/95/3...,A02,A0204,A02040400,제주특별자치도 서귀포시 표선면 녹산로 464-78,126.734110,33.393937,제주도 서귀포시 동남쪽 끝에 위치한 표선면 가시리는 제주도 내에서 아직 개발되지 않...,4.7,(3),제주도 서귀포시 동남쪽 끝에 위치한 표선면 가시리는 제주도 내에서 아직 개발되지 않...,서귀포시 동남쪽 끝 위치 표선면 가시리 개발 지역 년 국내 최초 지역 주민 참여 방...
1,1,갈중이(천연염색체험),https://blog.kakaocdn.net/dn/Hh733/btrcA1znbC9...,A02,A0203,A02030200,제주특별자치도 서귀포시 사계남로216번길 24-61 갈중이,126.313658,33.235142,갈옷은 제주도에서 구하기 쉬운 감을 이용해 만든 옷을 말하며 제주에서 일상복으로 즐...,4.6,(7),갈옷은 제주도에서 구하기 쉬운 감을 이용해 만든 옷을 말하며 제주에서 일상복으로 즐...,갈옷 구 쉽다 감 이용 옷 말 상복 전통 의상 갈옷 자외선 차단 효과 통 기성 뛰어...
2,2,감귤박물관,http://tong.visitkorea.or.kr/cms/resource/68/2...,A02,A0206,A02060100,제주특별자치도 서귀포시 효돈순환로 441,126.608090,33.271182,"서귀포 감귤 박물관은 제주 특산물인 감귤을 테마로 감귤의 세계를 한눈에 보고, 느끼...",4.0,"(1,475)",서귀포 감귤 박물관은 제주 특산물인 감귤을 테마로 감귤의 세계를 한눈에 보고 느끼고...,서귀포 감귤 박물관 특산 물 감귤 테마 감귤 세계 한눈 보고 체험 개관 공립 전문 ...
3,3,감사공묘역,http://tong.visitkorea.or.kr/cms/resource/96/3...,A02,A0201,A02010700,제주특별자치도 제주시 조천읍 함대로 362,126.661147,33.512995,조천읍 함덕리에 있는 감사공묘역은 조선 초기 한림학사와 전라감사를 역임한 영암 강영...,5.0,(6),조천읍 함덕리에 있는 감사공묘역은 조선 초기 한림학사와 전라감사를 역임한 영암 강영...,조천읍 함덕 감사 묘역 조선 초기 한림 학사 전라 감사 역임 영암 강영 선생 묘역 ...
4,4,감수굴 밭담길,http://tong.visitkorea.or.kr/cms/resource/82/3...,A02,A0203,A02030100,제주특별자치도 제주시 구좌읍 평대7길 34,126.843281,33.531057,평대리 중동에 위치한 감수굴 밭담길은 고즈넉한 마을의 옛 정취 그대로의 풍경을 즐기...,4.5,(2),평대리 중동에 위치한 감수굴 밭담길은 고즈넉한 마을의 옛 정취 그대로의 풍경을 즐기...,평 대리 중동 위치 감수 굴 밭 넉 마을 정취 풍경 산책 길이 평 대리 중동 마을 ...


In [33]:
keyword_input_mat = tfidf_vectorizer.transform([(keyword_input)])
keyword_input_sim = cosine_similarity(keyword_input_mat, tfidf_matrix)
keyword_input_ind = keyword_input_sim.argsort()[:, ::-1]
result_df = df.iloc[keyword_input_ind[0]]
print(keyword_input_ind)
print(result_df)

[[1874 1938 1877 ... 1300 1301    0]]
      Unnamed: 0         title  \
1874        2144        에어그라운드   
1938        2211  햇살정원 제주캠핑&민박   
1877        2147        에코힐글램핑   
1875        2145       에코그린캠핑장   
1867        2137       숲속야영장휴림   
...          ...           ...   
1298        1416           가파도   
1299        1417        각시바우오름   
1300        1421           강정천   
1301        1422          강정포구   
0              0  가시리국산화풍력발전단지   

                                             firstimage cat1   cat2  \
1874  http://tong.visitkorea.or.kr/cms/resource/71/2...  A03  A0302   
1938  http://tong.visitkorea.or.kr/cms/resource/52/2...  A03  A0302   
1877  http://tong.visitkorea.or.kr/cms/resource/97/2...  A03  A0302   
1875  http://tong.visitkorea.or.kr/cms/resource/32/2...  A03  A0302   
1867  http://tong.visitkorea.or.kr/cms/resource/65/2...  A03  A0302   
...                                                 ...  ...    ...   
1298  https://blog.kakaocdn.net/dn/Hh733/btrcA1znbC9... 

In [36]:
filtered_df = result_df[result_df.apply(lambda x: lat_long_distance(x['mapy'], x['mapx'], user_latitude, user_longitude) <= 15, axis=1)]
print(filtered_df)

      Unnamed: 0    title                                         firstimage  \
1176        1289     평대스낵  http://tong.visitkorea.or.kr/cms/resource/71/2...   
609          702    목화휴게소  http://tong.visitkorea.or.kr/cms/resource/93/2...   
627          720    미엘드세화  http://tong.visitkorea.or.kr/cms/resource/55/2...   
656          752   백기해녀의집  http://tong.visitkorea.or.kr/cms/resource/07/1...   
632          725  바다는안보여요  http://tong.visitkorea.or.kr/cms/resource/97/1...   
...          ...      ...                                                ...   
1216        1330    항아리조림  http://tong.visitkorea.or.kr/cms/resource/11/2...   
1218        1332   해녀의 부엌  https://blog.kakaocdn.net/dn/Hh733/btrcA1znbC9...   
1228        1342       해왓  http://tong.visitkorea.or.kr/cms/resource/58/2...   
1309        1431    검멀레해변  http://tong.visitkorea.or.kr/cms/resource/12/1...   
1316        1439    광치기해변  http://tong.visitkorea.or.kr/cms/resource/55/2...   

     cat1   cat2       cat3            

In [16]:
def rec_address(request):
    if request.method == 'POST':

        # 사용자가 입력한 정보 가져오기
        user_input = user_input
        user_address = user_address
        keyword_input = tokenize_user_input(user_input)
        keyword_input = ' '.join(keyword_input)

        # 주소
        user_coordinates = get_address_info(user_address)
        user_latitude = user_coordinates['Latitude'].iloc[0]
        user_longitude = user_coordinates['Longitude'].iloc[0]

        # 데이터 가져오기
        file_path = os.path.join(settings.BASE_DIR, 'jeju_olleh', 'modules', 'fin_token.csv')
        df = pd.read_csv(file_path, encoding='utf-8')

        # TF-IDF matrix, vectorizer 가져오기
        tfidf_mat = load_tfidf_matrix()
        tfidf_vect = load_tfidf_vectorizer()

        # 사용자 입력 키워드에 대한 TF-IDF 값 계산
        keyword_input_mat = tfidf_vect.transform([(keyword_input)])
        keyword_input_sim = cosine_similarity(keyword_input_mat, tfidf_mat)
        keyword_input_ind = keyword_input_sim.argsort()[:, ::-1]
        
        result_df = df.iloc[keyword_input_ind[0]]

        # result_df['selected_image_url'] = result_df['title'].apply(get_image_url)

        filtered_df = result_df[result_df.apply(lambda x: lat_long_distance(x['mapx'], x['mapy'], user_latitude, user_longitude) <= 15, axis=1)]

        recommended_places = filtered_df.head(5)

        return render(request, 'rec_app/other.html', {'user_longitude': user_longitude, 'user_latitude': user_latitude })
    return render(request, 'rec_app/rec_input.html')
